# 🤖 Project Triad: Trembling Hand & Strategic Robustness

This notebook runs the experiments for the paper **"Trembling Hands and Hidden Coalitions"**.
It benchmarks LLM agents in 3-player games (PGG, Volunteer's Dilemma, Triadic PD) to analyze social intelligence and robustness against noise.

**Repository:** [https://github.com/technoob05/Trembling-Triads](https://github.com/technoob05/Trembling-Triads)

## 1. Setup Environment
Clone the repository and install dependencies.

In [ ]:
!git clone https://github.com/technoob05/Trembling-Triads.git
%cd Trembling-Triads
!pip install -r requirements.txt

## 2. Hugging Face Login (Optional)
Required if you want to use gated models like **Llama-3-70B** or **Llama-3-8B**.
Make sure to add your HF Token to Kaggle Secrets with key `HF_TOKEN`.

In [ ]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HF_TOKEN")
    login(token=hf_token)
    print("Logged in to Hugging Face successfully!")
except Exception as e:
    print(f"Skipping login: {e}")

## 3. Experiment A: The Baseline (Mock vs Llama-3)
Run a quick test to verify the pipeline. 
*   **Game:** Public Goods Game (PGG)
*   **Models:** MockModel (Random) vs Llama3-8B
*   **Language:** Vietnamese & English

In [ ]:
# Run Benchmark
# Note: Change --models to 'Llama3-8B' or 'Llama3-70B' if GPU is available

!python triad_experiment.py --game PGG --models MockModel --languages vn,en --rounds 5

## 4. Experiment B: Volunteer's Dilemma test
Testing the new game mechanics.

In [ ]:
!python triad_experiment.py --game VD --models MockModel --rounds 5

## 5. Experiment C: Trembling Hand (Noise Injection)
Injecting 10% noise (`--noise 0.1`) to test robustness.
Does the agent retaliate immediately or forgive?

In [ ]:
!python triad_experiment.py --game PGG --noise 0.1 --models MockModel --rounds 10

## 6. View Results
List generated JSON files and inspect one.

In [ ]:
import os
import json

# List result files
result_files = [f for f in os.listdir('.') if f.endswith('.json')]
print("Result Files:", result_files)

# Read the latest one
if result_files:
    latest_file = max(result_files, key=os.path.getctime)
    with open(latest_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(json.dumps(data, indent=2, ensure_ascii=False)[:1000] + "...\n(truncated)")